# Intel® Extension for Scikit-learn Ridge Regression for IEEE-CIS Fraud Detection dataset

In [1]:
import pandas as pd
from timeit import default_timer as timer
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from IPython.display import HTML

warnings.filterwarnings("ignore")

### Download the data

In [2]:
dataset = fetch_openml(data_id=45955, as_frame=True)
data = dataset.frame

### Preprocessing
Let's encode categorical features with LabelEncoder

In [3]:
data = dataset.frame

# Simple imputation for missing values
data = data.fillna(0)

# Set the target variable
x = data.drop(columns=['fraud'])
y = data['fraud']

In [4]:
for col in ['used_pin_number', 'online_order', 'used_chip', 'repeat_retailer']:
    le = LabelEncoder().fit(x[col])
    x[col] = le.transform(x[col])

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((900000, 7), (100000, 7), (900000,), (100000,))

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_x = MinMaxScaler()
scaler_y = StandardScaler()

In [7]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1, 1)

scaler_x.fit(x_train)
x_train = scaler_x.transform(x_train)
x_test = scaler_x.transform(x_test)

scaler_y.fit(y_train)
y_train = scaler_y.transform(y_train).ravel()
y_test = scaler_y.transform(y_test).ravel()

### Patch original Scikit-learn with Intel® Extension for Scikit-learn
Intel® Extension for Scikit-learn (previously known as daal4py) contains drop-in replacement functionality for the stock Scikit-learn package. You can take advantage of the performance optimizations of Intel® Extension for Scikit-learn by adding just two lines of code before the usual Scikit-learn imports:

In [8]:
from sklearnex import patch_sklearn

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [9]:
from sklearn.linear_model import Ridge

params = {
    "alpha": 0.3,
    "fit_intercept": False,
    "random_state": 0,
    "copy_X": False,
}
start = timer()
model = Ridge(random_state=0).fit(x_train, y_train)
train_patched = timer() - start
f"Intel® extension for Scikit-learn time: {train_patched:.2f} s"

'Intel® extension for Scikit-learn time: 0.03 s'

In [10]:
y_predict = model.predict(x_test)
mse_metric_opt = metrics.mean_squared_error(y_test, y_predict)
f"Patched Scikit-learn MSE: {mse_metric_opt}"

'Patched Scikit-learn MSE: 0.6899813655698377'

### Train the same algorithm with original Scikit-learn
In order to cancel optimizations, we use *unpatch_sklearn* and reimport the class Ridge

In [11]:
from sklearnex import unpatch_sklearn

unpatch_sklearn()

In [12]:
from sklearn.linear_model import Ridge

start = timer()
model = Ridge(random_state=0).fit(x_train, y_train)
train_unpatched = timer() - start
f"Original Scikit-learn time: {train_unpatched:.2f} s"

'Original Scikit-learn time: 0.12 s'

In [13]:
y_predict = model.predict(x_test)
mse_metric_original = metrics.mean_squared_error(y_test, y_predict)
f"Original Scikit-learn MSE: {mse_metric_original}"

'Original Scikit-learn MSE: 0.6899813655698382'

In [14]:
HTML(
    f"<h3>Compare MSE metric of patched Scikit-learn and original</h3>"
    f"MSE metric of patched Scikit-learn: {mse_metric_opt} <br>"
    f"MSE metric of unpatched Scikit-learn: {mse_metric_original} <br>"
    f"Metrics ratio: {mse_metric_opt/mse_metric_original} <br>"
    f"<h3>With Scikit-learn-intelex patching you can:</h3>"
    f"<ul>"
    f"<li>Use your Scikit-learn code for training and prediction with minimal changes (a couple of lines of code);</li>"
    f"<li>Fast execution training and prediction of Scikit-learn models;</li>"
    f"<li>Get the similar quality</li>"
    f"<li>Get speedup in <strong>{(train_unpatched/train_patched):.1f}</strong> times.</li>"
    f"</ul>"
)